## 텐서플로 : 수치 계산을 위한 강력한 오픈소스 소프트웨어 라이브러리

### 1. 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [37]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2 #계산하는 것 같아 보이지만 실제로 어떤 계산도 수행하지 않는다. 단지, 계산 그래프만 만들 뿐!

In [3]:
#계산 그래프를 평가하려면 텐서플로 세션 시작
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

42


In [8]:
#매번 sess.run()을 반복하기 번거롭다면
with tf.Session() as sess:
    x.initializer.run() #tf.get_default_session().run(x.initializer) 와 동일
    y.initializer.run()
    result = f.eval() #tf.get_default_session().run(f) 와 동일
    print(result)

sess.close()

42


In [9]:
#global_variables_initializer() : 각 변수의 초기화 : 계산 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성
init = tf.global_variables_initializer() #init 노드 준비
with tf.Session() as sess:
    init.run() #실제 모든 변수 초기화
    result = f.eval()
    print(result)

sess.close()

42


#### 텐서플로 : (구성 단계 - 계산 그래프 만들기)   (실행 단계 - 그래프 실행하기)

### 2. 계산 그래프 관리

In [10]:
#노드를 만들면 자동으로 기본 계산 그래프에 추가
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
#독립적인 계산 그래프를 여러 개 만들어야할 때, 새로운 graph 객체 만들어 with 블록 안에서 사용
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

### 3. 노드 값의 생애주기

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval()) #10 : y가 x에 의존하고 x가 w에 의존하기 때문에 w를 평가하고 그 다음에 x,y를 평가
    print(z.eval()) #15 : z가 x에 의존하고 x가 w에 의존하기 때문에 w를  평가하고 그다음에 x,z를 평가

# 위의 코드는 w와 x를 두 번 평가

10
15


In [14]:
#w와 x를 두 번 평가하지 않고 y와 z를 효율적으로 평가하는 방법
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


### 4. 텐서플로를 이용한 선형 회귀

In [16]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = (tf.matrix_inverse(XT @ X) @ XT) @ y

with tf.Session() as sess:
    theta_value = theta.eval()

### 5. 경사 하강법 구현

In [19]:
#직접 그래디언트 계산

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')

In [20]:
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch',epoch,'MSE =',mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 10.920551
Epoch 100 MSE = 4.974613
Epoch 200 MSE = 4.880268
Epoch 300 MSE = 4.8625784
Epoch 400 MSE = 4.8498487
Epoch 500 MSE = 4.840043
Epoch 600 MSE = 4.83244
Epoch 700 MSE = 4.8265142
Epoch 800 MSE = 4.8218713
Epoch 900 MSE = 4.818216


In [38]:
#자동 미분 사용

reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# auto-gradients
gradients = tf.gradients(mse, [W])[0]
training_op = tf.assign(W, W - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)

print(best_W)

epoch: 000, MSE = 8.440990
epoch: 100, MSE = 4.905502
epoch: 200, MSE = 4.844091
epoch: 300, MSE = 4.834477
epoch: 400, MSE = 4.827744
epoch: 500, MSE = 4.822569
epoch: 600, MSE = 4.818562
epoch: 700, MSE = 4.815442
epoch: 800, MSE = 4.813001
epoch: 900, MSE = 4.811080
[[-0.1673944 ]
 [ 0.88740253]
 [ 0.14401646]
 [-0.34770864]
 [ 0.36178356]
 [ 0.00393806]
 [-0.04269556]
 [-0.6614535 ]
 [-0.6375285 ]]


In [39]:
#옵티마이저 사용

reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)

print(best_W)

epoch: 000, MSE = 8.440990
epoch: 100, MSE = 4.905502
epoch: 200, MSE = 4.844091
epoch: 300, MSE = 4.834477
epoch: 400, MSE = 4.827744
epoch: 500, MSE = 4.822569
epoch: 600, MSE = 4.818562
epoch: 700, MSE = 4.815442
epoch: 800, MSE = 4.813001
epoch: 900, MSE = 4.811080
[[-0.1673944 ]
 [ 0.88740253]
 [ 0.14401646]
 [-0.34770864]
 [ 0.36178356]
 [ 0.00393806]
 [-0.04269556]
 [-0.6614535 ]
 [-0.6375285 ]]


In [24]:
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum=0.9)

### 6. 훈련 알고리즘에 데이터 주입

In [27]:
#placeholder : 실제로 아무 계산도 하지 않는 특수 노드

A = tf.placeholder(tf.float32, shape=(None,3)) #None : 어떤 크기도 가능
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
    
print(B_val_1)
print()
print(B_val_2)

[[6. 7. 8.]]

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [40]:
reset_graph()

# placeholder setting
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")  # (None, 9)
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

################
# hyper-params #
################
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))  # (20640/100) 올림

# mini-batch random sampling
def fetch_batch(epoch, batch_index, batch_size):
    """ epoch당 batch_size만큼 Random sampling 하는 메소드
    
    Args:
        - epoch: training epoch
        - batch_index: batch steps
        - batch_size: batch size
    
    Returns:
        - X_batch, y_batch: batch size만큼의 X, y 데이터셋
    """    
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_W = W.eval()
    
print(best_W)

[[-0.1673944 ]
 [ 0.86515963]
 [ 0.11857292]
 [-0.29859844]
 [ 0.39628983]
 [-0.00649126]
 [ 0.00409027]
 [-0.8007697 ]
 [-0.7800274 ]]


### 7. 모델 저장과 복원

In [41]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y

# loss function
mse = tf.reduce_mean(tf.square(error), name='mse')
# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=3)
tf.add_to_collection('weight', W)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
            save_path = saver.save(sess, './model/my_model.ckpt')
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)
    save_path = saver.save(sess, "./model/my_model_final.ckpt")

print(best_W)

epoch: 000, MSE = 8.440990
epoch: 100, MSE = 4.905502
epoch: 200, MSE = 4.844091
epoch: 300, MSE = 4.834477
epoch: 400, MSE = 4.827744
epoch: 500, MSE = 4.822569
epoch: 600, MSE = 4.818562
epoch: 700, MSE = 4.815442
epoch: 800, MSE = 4.813001
epoch: 900, MSE = 4.811080
[[-0.1673944 ]
 [ 0.88740253]
 [ 0.14401646]
 [-0.34770864]
 [ 0.36178356]
 [ 0.00393806]
 [-0.04269556]
 [-0.6614535 ]
 [-0.6375285 ]]


In [42]:
with tf.Session() as sess:
    saver.restore(sess, './model/my_model_final.ckpt')
    best_W_restored = tf.get_collection('weight')[0]
    best_W_restored = best_W_restored.eval()
    
# compare element-wise equal or not
print(np.allclose(best_W, best_W_restored))  # True

W0821 21:59:34.972602 140154987943680 deprecation.py:323] From /home/pirl/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


True


### 8. 텐서보드로 그래프와 학습 곡선 시각화하기

In [43]:
# TensorFlow Graph visualizer code
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [44]:
show_graph(tf.get_default_graph())

In [48]:
import graphviz
import tfgraphviz as tfg

tf_graph = tfg.board(tf.get_default_graph())
tf_graph